In [2]:
from shallowstack.neural_net.datamodule import PokerDataModule
from shallowstack.state_manager.state_manager import PokerGameStage
import torch


data = PokerDataModule(PokerGameStage.RIVER, 5, 1000)
data.setup("")

i = iter(data.train_dataloader())
value: torch.Tensor = next(i)

In [4]:
data2 = PokerDataModule(PokerGameStage.TURN, 5, 1000)
data2.setup("")

i2 = iter(data2.train_dataloader())
value2: torch.Tensor = next(i2)

In [9]:
value2

tensor([[[ 7.0272e-04,  3.0481e-04,  0.0000e+00,  ...,  2.8445e-09,
          -2.0390e-09,  0.0000e+00]],

        [[ 1.5994e-04,  1.3040e-03,  1.3403e-03,  ..., -7.2889e-09,
          -9.9699e-11,  0.0000e+00]],

        [[ 1.1677e-03,  5.8479e-04,  1.3240e-03,  ..., -8.9971e-09,
           2.4947e-09,  0.0000e+00]],

        [[ 1.4418e-03,  7.3638e-04,  4.7491e-04,  ...,  9.0962e-09,
           8.4115e-09,  0.0000e+00]],

        [[ 0.0000e+00,  1.1319e-03,  8.3868e-04,  ...,  7.5815e-10,
          -3.3374e-09,  0.0000e+00]]], dtype=torch.float64)

In [8]:
torch.device("mps")

device(type='mps')

In [7]:
x, y = value.split([1326 * 2 + 1 + 5, 1326 * 2 + 1], dim=1)
print(x.dtype)

print(x.shape)
print(y.shape)

torch.float32
torch.Size([5, 2658])
torch.Size([5, 2653])


In [4]:
from lightning import Trainer
from shallowstack.neural_net.model import ValueNetwork


model = ValueNetwork(1326, 5)


In [5]:
print(x[:, :1326] @ y[:, :1326].T)
print(torch.dot(x[0, :1326], y[0, :1326]))

tensor([[-2.5402e-04, -4.3441e-05, -1.0448e-04, -2.8454e-04, -1.1668e-04],
        [-2.5339e-04, -4.3337e-05, -1.0422e-04, -2.8383e-04, -1.1640e-04],
        [-2.5312e-04, -4.3288e-05, -1.0411e-04, -2.8353e-04, -1.1627e-04],
        [-2.5534e-04, -4.3667e-05, -1.0502e-04, -2.8602e-04, -1.1729e-04],
        [-2.5691e-04, -4.3936e-05, -1.0567e-04, -2.8777e-04, -1.1802e-04]])
tensor(-0.0003)


In [6]:
x.shape

torch.Size([5, 2658])

In [7]:
torch.bmm(x[:, :1326*2 + 1].unsqueeze(1), y.unsqueeze(2)).squeeze(1).shape

torch.Size([5, 1])

In [8]:
model(x)

tensor([[ 0.0259,  0.0560, -0.1152,  ..., -0.1321,  0.1834,  0.0051],
        [ 0.0346,  0.0225, -0.1206,  ..., -0.1261,  0.1912, -0.0002],
        [ 0.0269,  0.0417, -0.1126,  ..., -0.1403,  0.1918,  0.0029],
        [ 0.0341,  0.0579, -0.1223,  ..., -0.1287,  0.1923,  0.0025],
        [ 0.0263,  0.0402, -0.1252,  ..., -0.1299,  0.1774,  0.0069]],
       grad_fn=<CatBackward0>)

In [13]:

trainer = Trainer(max_epochs=200)
trainer.fit(model, data)

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name         | Type   | Params
----------------------------------------
0 | fc1          | Linear | 680 K 
1 | fc2          | Linear | 32.9 K
2 | fc3          | Linear | 8.3 K 
3 | fc4          | Linear | 2.1 K 
4 | value_output | Linear | 87.5 K
----------------------------------------
811 K     Trainable params
0         Non-trainable params
811 K     Total params
3.246     Total estimated model params size (MB)


Epoch 199: 100%|██████████| 3/3 [00:00<00:00, 96.05it/s, v_num=2] 

`Trainer.fit` stopped: `max_epochs=200` reached.


Epoch 199: 100%|██████████| 3/3 [00:00<00:00, 58.60it/s, v_num=2]
